In [3]:
import ctypes

user32_lib = ctypes.WinDLL(r"C:\Windows\System32\user32.dll")
kernel32_lib = ctypes.WinDLL(r"C:\Windows\System32\kernel32.dll")


# Lowest level struct from msdn docs

In [6]:
LONG = ctypes.c_long
DWORD = ctypes.c_ulong
ULONG_PTR = ctypes.POINTER(DWORD)
WORD = ctypes.c_ushort

STRUCT = ctypes.Structure
UNION = ctypes.Union


class tagMOUSEINPUT(STRUCT):
    _fields_ = (('dx', LONG),
                ('dy', LONG),
                ('mouseData', DWORD),
                ('dwFlags', DWORD),
                ('time', DWORD),
                ('dwExtraInfo', ULONG_PTR))


class tagKEYBDINPUT(STRUCT):
    _fields_ = (('wVk', WORD),
                ('wScan', WORD),
                ('dwFlags', DWORD),
                ('time', DWORD),
                ('dwExtraInfo', ULONG_PTR))


class tagHARDWAREINPUT(STRUCT):
    _fields_ = (('uMsg', DWORD),
                ('wParamL', WORD),
                ('wParamH', WORD))


class Union_tagINPUT(UNION):
    _fields_ = (('mi', tagMOUSEINPUT),
                ('ki', tagKEYBDINPUT),
                ('hi', tagHARDWAREINPUT))


class tagINPUT(STRUCT):
    _fields_ = (('type', DWORD),
                ('dummy_union', Union_tagINPUT))


#Wrap method for convenient

In [ ]:
# see https://docs.microsoft.com/en-us/windows/desktop/api/winuser/ns-winuser-tagmouseinput#members for details.
# mouseData for tagMOUSEINPUT struct
WHEEL_DELTA = 120
XBUTTON1 = 0x0001
XBUTTON2 = 0x0002

# dwFlags for tagMOUSEINPUT struct
MOUSEEVENTF_ABSOLUTE = 0x8000
MOUSEEVENTF_HWHEEL = 0x01000
MOUSEEVENTF_MOVE = 0x0001
MOUSEEVENTF_MOVE_NOCOALESCE = 0x2000
MOUSEEVENTF_LEFTDOWN = 0x0002
MOUSEEVENTF_LEFTUP = 0x0004
MOUSEEVENTF_RIGHTDOWN = 0x0008
MOUSEEVENTF_RIGHTUP = 0x0010
MOUSEEVENTF_MIDDLEDOWN = 0x0020
MOUSEEVENTF_MIDDLEUP = 0x0040
MOUSEEVENTF_VIRTUALDESK = 0x4000
MOUSEEVENTF_WHEEL = 0x0800
MOUSEEVENTF_XDOWN = 0x0080
MOUSEEVENTF_XUP = 0x0100

# type for tagINPUT struct
INPUT_MOUSE = 0
INPUT_KEYBOARD = 1
INPUT_HARDWARE = 2

"""
def make_mouseinput(data, flags, x, y):
    return tagMOUSEINPUT(x, y, data, flags, 0, None)
"""


def make_mouse_input(data, flags, x, y):
    return tagINPUT(INPUT_MOUSE, Union_tagINPUT(mi=tagMOUSEINPUT(x, y, data, flags, 0, None)))


def send_input(*inputs):
    n_inputs = len(inputs)
    lpointer_input = tagINPUT * n_inputs
    pointer_inputs = lpointer_input(*inputs)
    cb_size = ctypes.c_int(ctypes.sizeof(tagINPUT))
    return ctypes.windll.user32.SendInput(n_inputs, pointer_inputs, cb_size)


1

# Normalize Coordinates


In [12]:
def normalize_coord(x, y):
    screen_xsize, screen_ysize = user32_lib.GetSystemMetrics(0), user32_lib.GetSystemMetrics(1)
    return int((x / screen_xsize) * 65535), int((y / screen_ysize) * 65535)


In [14]:
from win_api import most_wanted_cursor_pos
from special_wnds import SpeedMeter

send_input(make_mouse_input(XBUTTON2, 0x8019, *normalize_coord(*most_wanted_cursor_pos(SpeedMeter()))))


1